In [1]:
import osmnx as ox
import pandas as pd

ox.settings.use_cache = True
ox.settings.log_console = True

# Import Module


df_store = pd.read_excel("data/demand_and_supply.xlsx", sheet_name="Store")
df_terminal = pd.read_excel("data/demand_and_supply.xlsx", sheet_name="Terminal")

df_plan = pd.read_csv("data/demand_and_supply_plan.csv")
df_plan = df_plan[df_plan["supply plan"] != 0]

df = pd.merge(df_plan, df_store, on="store", how="inner")
df = pd.merge(df, df_terminal, on="terminal", how="inner", suffixes=('_s', '_t'))

df["name"] = df["store"] + " - " + df["terminal"]
df["path"] = df.apply(lambda x: [[x["lat_s"], x["lon_s"]], [x["lat_t"], x["lon_t"]]], axis=1)
# print(df)

route_df = df[["name", "path"]]
route_df["color"] = route_df.apply(lambda x: tuple(int("ffe800"[i: i + 2], 16) for i in (0, 2, 4)), axis=1)

print(route_df)

# get graph and add free-flow travel times to its edges
G = ox.graph_from_place("Bengaluru, Karnataka, India", network_type="drive")
G = ox.add_edge_travel_times(ox.add_edge_speeds(G))




C:\Users\manyu\AppData\Local\Temp\ipykernel_7100\339651662.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_df["color"] = route_df.apply(lambda x: tuple(int("ffe800"[i: i + 2], 16) for i in (0, 2, 4)), axis=1)


         name                                               path  \
0   AZ - West  [[12.9359658083896, 77.6113117509235], [12.914...   
1  GA - North  [[12.9091751039833, 77.6005861284724], [12.945...   
2  GA - South  [[12.9091751039833, 77.6005861284724], [12.892...   
3   GA - West  [[12.9091751039833, 77.6005861284724], [12.914...   
4   NJ - East  [[12.9293772293161, 77.5913562480048], [12.920...   
5  NJ - North  [[12.9293772293161, 77.5913562480048], [12.945...   

           color  
0  (255, 232, 0)  
1  (255, 232, 0)  
2  (255, 232, 0)  
3  (255, 232, 0)  
4  (255, 232, 0)  
5  (255, 232, 0)  


In [9]:
df_store = pd.read_excel("data/demand_and_supply.xlsx", sheet_name="Store")
df_terminal = pd.read_excel("data/demand_and_supply.xlsx", sheet_name="Terminal")

df_plan = pd.read_csv("data/demand_and_supply_plan.csv")
df_plan = df_plan[df_plan["supply plan"] != 0]

df = pd.merge(df_plan, df_store, on="store", how="inner")
df = pd.merge(df, df_terminal, on="terminal", how="inner", suffixes=('_s', '_t'))

df["name"] = df["store"] + " - " + df["terminal"]
df["path"] = df.apply(lambda x: [[x["lat_s"], x["lon_s"]], [x["lat_t"], x["lon_t"]]], axis=1)
# print(df)

route_df = df[["name", "path"]]
route_df["color"] = route_df.apply(lambda x: tuple(int("ffe800"[i: i + 2], 16) for i in (0, 2, 4)), axis=1)

route_df

C:\Users\manyu\AppData\Local\Temp\ipykernel_7100\1715907584.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_df["color"] = route_df.apply(lambda x: tuple(int("ffe800"[i: i + 2], 16) for i in (0, 2, 4)), axis=1)


,name,path,color
0,AZ - West,"[[12.9359658083896, 77.6113117509235], [12.914...","(255, 232, 0)"
1,GA - North,"[[12.9091751039833, 77.6005861284724], [12.945...","(255, 232, 0)"
2,GA - South,"[[12.9091751039833, 77.6005861284724], [12.892...","(255, 232, 0)"
3,GA - West,"[[12.9091751039833, 77.6005861284724], [12.914...","(255, 232, 0)"
4,NJ - East,"[[12.9293772293161, 77.5913562480048], [12.920...","(255, 232, 0)"
5,NJ - North,"[[12.9293772293161, 77.5913562480048], [12.945...","(255, 232, 0)"


In [10]:
def find_shortest_route(_path):
    station_org_node_id = ox.distance.nearest_nodes(G, [_path[0][1]], [_path[0][0]])
    station_dst_node_id = ox.distance.nearest_nodes(G, [_path[1][1]], [_path[1][0]])

    paths = ox.shortest_path(G, station_org_node_id, station_dst_node_id, weight="travel_time", cpus=None)

    paths_latlng = [[(G.nodes[node]["y"], G.nodes[node]["x"]) for node in path] for path in paths]
    return paths_latlng

In [11]:
route_df["path"] = route_df["path"].apply(find_shortest_route)
route_df.to_json("data/optimal_routes.json")

C:\Users\manyu\AppData\Local\Temp\ipykernel_7100\3816485105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_df["path"] = route_df["path"].apply(find_shortest_route)


In [4]:
route_df["path"]

0    [[(13.1423666, 77.5806212)]]
1    [[(13.1423666, 77.5806212)]]
2    [[(13.1423666, 77.5806212)]]
3    [[(13.1423666, 77.5806212)]]
4    [[(13.1423666, 77.5806212)]]
5    [[(13.1423666, 77.5806212)]]
Name: path, dtype: object

In [7]:
find_shortest_route([ [12.9359658083896, 77.6113117509235], [12.9091751039833, 77.6005861284724]])

[[(12.9369075, 77.6110193),
  (12.9344528, 77.6123562),
  (12.9343686, 77.6123887),
  (12.9324651, 77.6134413),
  (12.9316547, 77.6138662),
  (12.9313406, 77.6140453),
  (12.9298694, 77.6149477),
  (12.9295976, 77.6151403),
  (12.9295314, 77.6151899),
  (12.9295061, 77.6150797),
  (12.9302495, 77.612799),
  (12.9308676, 77.611721),
  (12.9310592, 77.6104049),
  (12.9311031, 77.6098614),
  (12.9309168, 77.6098163),
  (12.9309579, 77.6094364),
  (12.9309594, 77.6093424),
  (12.9309667, 77.609197),
  (12.930973, 77.6090984),
  (12.9309863, 77.6088067),
  (12.9310569, 77.6082195),
  (12.9307476, 77.608228),
  (12.9306858, 77.6077072),
  (12.9306779, 77.6075862),
  (12.9306236, 77.6071885),
  (12.9305686, 77.6067779),
  (12.9303701, 77.6067889),
  (12.9301084, 77.6067951),
  (12.930092, 77.6067937),
  (12.9298225, 77.6067625),
  (12.9295845, 77.6066679),
  (12.9293081, 77.6065659),
  (12.9290878, 77.6065048),
  (12.9284991, 77.6063671),
  (12.9282449, 77.6063348),
  (12.9279795, 77.6062921)